# SuperSearch Solution Frontend


# 0. Installing and loading libraries

In [1]:
!pip install google-cloud-aiplatform scann gradio --quiet


In [2]:
# colab auth
from google.colab import auth
auth.authenticate_user()

In [3]:
#Setting project environment
PROJECT_ID = "capstoneie"  #ADD PROJECT_ID:
LOCATION = "us-central1"  # Choose location.


#Google's GenAI libraries in GCP
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

from google.api_core import retry
from vertexai.language_models import TextEmbeddingModel


#Additional libraries
from tqdm.auto import tqdm
tqdm.pandas()
import time
import pandas as pd
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import numpy as np


#libraries for Search and frontend
import scann
import gradio as gr
import ast


# 1.Loading Files

In [4]:
df_genAI =pd.read_csv("/content/df1_genAI_juices_100_items_and_embeddings.csv")
df_genAI = df_genAI.drop(columns='Unnamed: 0', axis=1)

df_genAI.head(1)

id_producto                nombre_producto departamento           pasillo  \
0            7  PURA AGUA DE COCO CON NARANJA      BEBIDAS  NÉCTARES DE JUGO   

                                                                                                                                                                                                                                                                                       answer  \
0  Disfruta del refrescante y vibrante sabor de nuestra agua de coco con naranja.  La combinación perfecta entre la dulzura del coco y la frescura cítrica de la naranja. Rehidrátate con esta bebida 100% natural, sin azúcar añadida. Ideal para disfrutar en cualquier momento del día. \n   

                                                                        usage_metadata  \
0  {'prompt_token_count': 241, 'candidates_token_count': 68, 'total_token_count': 309}   

   finish_reason  \
0            1.0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   safety_settings  \
0  [category: HARM_CATEGORY_HATE_SPEECH\nprobability: NEGLIGIBLE\nprobability_score: 0.09929540008306503\nseverity: HARM_SEVERITY_NEGLIGIBLE\nseverity_score: 0.13006427884101868\n, category: HARM_CATEGORY_DANGEROUS_CONTENT\nprobability: NEGLIGIBLE\nprobability_score: 0.19682618975639343\nseverity: HARM_SEVERITY_NEGLIGIBLE\nseverity_score: 0.19821932911872864\n, category: HARM_CATEGORY_HARASSMENT\nprobability: NEGLIGIBLE\nprobability_score: 0.17175468802452087\nseverity: HARM_SEVERITY_NEGLIGIBLE\nseverity_score: 0.10800065845251083\n, category: HARM_CATEGORY_SEXUALLY_EXPLICIT\nprobability: NEGLIGIBLE\nprobability_score: 0.32734566926956177\nseverity: HARM_SEVERITY_NEGLIGIBLE\nseverity_score: 0.16398026049137115\n]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

# 2. Search Engine

In [6]:
#Embegging model
embedding_model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-preview-0409")

In [7]:
# Retrieve embeddings from the specified model with retry logic
def make_embed_text_fn(model):
    @retry.Retry(timeout=300.0)
    def embed_fn(text):
        return embedding_model.get_embeddings([text])[0].values

    return embed_fn

In [8]:

# Convert string representations of lists to actual lists
def convert_to_list(s):
    try:
        return ast.literal_eval(s)
    except ValueError:
        return None

# Assuming df_genAI["embeddings_description"] is a pandas Series containing the string representations
embeddings_list = df_genAI["embeddings_description"].apply(convert_to_list).dropna().tolist()
record_count = len(embeddings_list)
dataset = np.array(embeddings_list)

# Normalize the dataset
normalized_dataset = dataset / np.linalg.norm(dataset, axis=1)[:, np.newaxis]

# Configure ScaNN as a tree - asymmetric hash hybrid with reordering
# Use scann.scann_ops.build() to create a TensorFlow-compatible searcher
searcher = (
    scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product")
    .tree(
        num_leaves=record_count,
        num_leaves_to_search=record_count,
        training_sample_size=record_count,
    )
    .score_ah(2, anisotropic_quantization_threshold=0.2)
    .reorder(100)
    .build()
)


# 3. Live App

SuperSearch App deployed in Gradio. Below we'll get a link where we can try the solution.

In [9]:
def search_function(query):
    start = time.time()
    query = embedding_model.get_embeddings([query])[0].values
    neighbors, distances = searcher.search(query, final_num_neighbors=3)
    end = time.time()

    products = []
    for id, dist in zip(neighbors, distances):
        products.append(f"{df_genAI.nombre_producto[int(id)][:125]} (Distance: {dist})")
    return "\n".join(products)

with gr.Blocks() as demo:
    gr.Markdown("""
    <style>
        .search-bar-container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f5f5f5;
        }
        .search-bar-box {
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0px 0px 15px rgba(0,0,0,0.1);
            width: 400px;
            text-align: center;
        }
        .search-bar-box h1 {
            font-size: 24px;
            margin-bottom: 20px;
        }
        .search-bar-box .gr-button {
            width: 100%;
            background-color: #007BFF;
            color: white;
        }
        .search-bar-box .gr-button:hover {
            background-color: #0056b3;
        }
    </style>
    """)

    with gr.Column(elem_id="search-bar-container"):
        with gr.Column(elem_id="search-bar-box"):
            gr.Markdown("<h1>SuperSearch: Next Gen Search</h1>")
            search_input = gr.Textbox(placeholder="Search for products...", label="Search")
            search_button = gr.Button("Search")
            output = gr.Textbox(label="Results", lines=5)

            search_button.click(search_function, inputs=search_input, outputs=output)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d834632e695a309554.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d834632e695a309554.gradio.live
